# FourSquare API with Python 

In this notebook we learn to use the <b>FourSquare API</b>.

FourSquare is a social location service that allows users to explore the world around them.

In [15]:
import pandas as pd
import numpy as np

<b>Folium</b> is a Python open sourse library for geospatial data visualization. Here we use it for plotting some selected locations on NYC map.

Using Folium, we can also specify the boundaries (as the coordinates of polygon vertices) of cities/counties/states/regions/countries/etc. via GeoJSON file, then plot a corresponding <b>choropleth map</b>: a map where those territories highlighted (e.g. colored by different colors) according to the value of some statistical variables (e.g. crime rate or an amount of greenery) associated with that region, given, say, as pandas dataframe. 

In [3]:
import folium
print("Folium is sucessfully imported!")

Folium is sucessfully imported!


<h3>geopy Library</h3>

The process of converting addresses (being it the address of a person or a business, or city/state/country name etc.) to geographic information — Latitude and Longitude — to map their locations is called <b>Geocoding</b>. The reverse process - look up addresses for a location (=given a latitude and longitude anywhere on the planet, find the nearest address) - is called <b>Reverse Geocoding</b>

<b>geopy</b> is a Python client for several popular geocoding web services, including <b>OpenStreetMap Nominatim</b> (which we are going to use here), Google Geocoding API (V3) and many others. 

In [4]:
import geopy
print("Geopy is sucessfully imported!")

Geopy is sucessfully imported!


In [5]:
from geopy.geocoders import Nominatim

Libraries for displaying images:

<h3>Credentials for connecting to FourSquare API</h3>

We need the <i>access token</i> for <b>user authentication</b>; for <b>userless authentication</b> <i>client_id</i> and <i>client_secret</i> provided on <u>Foursquare developers page</u> (https://foursquare.com/developers/apps, then click on your App - which obviously should be created first for that matter...), will suffice.

Note that some endpoints (e.g. <i>User Detail</i>) <b>require</b> user authentication! Also note that some other API calls (e.g. <i>Venue Tips</i>) return only <b>partial</b> information (part of users' credentials - like, say, user's id - is omitted) for some calls under userless autenticatiion. Hence, in order to make sure we get all the information - no matter what endpoint we're using, we can provide BOTH sets of credentials - <i>client_id</i> + <i>client_secret</i> AND access token - in each API call, regardless of this specific endpoint authentication requirements.

<b>We can get an access token as follows:</b>

(from https://developer.foursquare.com/docs/places-api/authentication/#userless-auth)

<b>Step 1</b>

Direct users to Foursquare with your registered redirect uri. On Foursquare developers page, I set my Redirect URL (aka YOUR_REGISTERED_REDIRECT_URI) to https://www.google.com (Don't forget to press "Save" after changing the Redirect URL).

https://foursquare.com/oauth2/authenticate?client_id=YOUR_CLIENT_ID&response_type=code&redirect_uri=YOUR_REGISTERED_REDIRECT_URI

<b>Step 2</b>

If the user accepts, they will be redirected back to your URI with a code.

https://YOUR_REGISTERED_REDIRECT_URI/?code=CODE

<b>Step 3</b>

Your server should exchange the code it got in step 2 for an access token.

https://foursquare.com/oauth2/access_token?client_id=YOUR_CLIENT_ID&client_secret=YOUR_CLIENT_SECRET&grant_type=authorization_code&redirect_uri=YOUR_REGISTERED_REDIRECT_URI&code=CODE
The response will be JSON.

{ "access_token": ACCESS_TOKEN }

<b>Step 4</b>

Once you have an access token, you can use any of the endpoints by adding oauth_token=ACCESS_TOKEN to your GET or POST request. For example, from the command line, you can do:

curl https://api.foursquare.com/v2/users/self/checkins?oauth_token=ACCESS_TOKEN&v=YYYYMMDD

In [158]:
CLIENT_ID = 'XXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXXX' # your Foursquare Secret
ACCESS_TOKEN = 'XXXXXXXXX' # your FourSquare Access Token

<h3>Real Life Situation</h3>

Let's say we stayed in NYC in Conrad Hotel. We would like to eat some Italian food, but don't feel like going long distances: maximum 500m. from the hotel room. We can get the addresses of all the Italian food restaurants located near by as follows:

(obviously we could just google it, but the following code represents what is going on "under the hood" when we do google something similar)

- First we create a Nominatim geocoder instance called "geolocator". We provide it with the address of the hotel we stayed in and obtain the hotel's coordinates.

Don't use a default user_agent!

<u>Query format in Nominatim:</u>

Free-form queries are processed first left-to-right and then right-to-left if that fails. So you may search for pilkington avenue, birmingham as well as for birmingham, pilkington avenue. Commas are optional, but improve performance by reducing the complexity of the search.

street = housenumber streetname

city = city
county = county
state = state
country = country
postalcode = postalcode

(see GeoPy documentation)

- Next, we provide the previously obtained latitude and longitude as part of a FourSquare API query, so that FourSquare knows in what area it supposed to search for the venues, and get all the relevant suggestions.
    
- Then, we choose the best one of the suggested places: we extract data for each venue using Venue Details endpoint and take a look on the place's rating.

- We also get some users' reviews via Venue Tips endpoint.

- Finally, just for the sake of practise, we get the detailed information about the user who left one of the tips and print this user's picture.


In [9]:
address = '102 North End Ave, New York, NY' # Conrad Hotel in NYC

geolocator = Nominatim(user_agent='FourSquare_rookie')
location = geolocator.geocode(address)

In [15]:
type(location)

geopy.location.Location

In [10]:
latitude = location.latitude
longitude = location.longitude

print(latitude,longitude)

40.7149555 -74.0153365


<h4>Important!</h4>

Note that up to this point we didn't use FourSquare API at all! You can extract the coordinates from the address by using an open source geocoders, like GeoPy, only. 

In [11]:
# Query date: we want to use the most up to date version of FourSquare API  
VERSION = '20210103'
# We limit the number of results per each query
LIMIT = 30

query = 'Italian'
radius = 500

In [14]:
# Import library for handling requests (e.g. sending a GET requests) 
import requests

<b>requests</b> is a Python package that allows to quickly and simply (no need to manually add query strings to your URLs, easily formatting query results - using .json, etc.) send a HTTP requests; note that <b>GET</b> request provided by this package has nothing to do <i>specifically</i> with FourSquare API and might be used to HTTP any web site. However, we do need it in order to make FourSquare API calls.

Note that in the following cell this is just a usual string formatting we previously used many times with print(...) function: obviously it can be used outside of print() too. 

<h3>Find the Desired Venues Nearby</h3>

In order to do that, we send a 'search' query. This is a <b>regular</b> FourSquare API endpoint.

After the basic URI "https://api.foursquare.com/v2/", we specify the <b>endpoint group</b> ('venues'), the <b>endpoint</b> ('search'; note the <b>'?'</b> sign after the endpoint), authentication details (client_id and client_secret, or authentication token), API version and all the additional information regarding the desired query, if any.

For detailed information regarding how to build URLs for FourSquare API calls, go to https://developer.foursquare.com/docs/places-api/endpoints/ (for endpoints information) and to https://developer.foursquare.com/docs/places-api/authentication/#userless-auth (for userless vs. user authentication)

In [16]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,VERSION, query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=BWN5AFL11Q15E005RDLMI4IL2YDPMGQILRXPMGDZJ10PWJOD&client_secret=KPYVQ0UIFPNWATAUU352BRTHN3RUVDN1C3THDDIHY3KBOKKE&ll=40.7149555,-74.0153365&v=20210103&query=Italian&radius=500&limit=30'

We create requests.Response object which contains query results and put these results into a .json file using .json() method

In [18]:
query_results = requests.get(url).json()
#query_results

You can check response fields for each endpoint on Foursquare API documentation page, <b>Endpoints overview</b> section.

In [19]:
query_results['response']['venues'][0]

{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351},
   {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
  'distance': 58,
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True}],
 'referralId': 'v-1610108733',
 'hasPerk': False}

In [20]:
query_results['response']['venues'][0].keys()

dict_keys(['id', 'name', 'location', 'categories', 'referralId', 'hasPerk'])

Next, we want to put all the relevant information (which is the list of Italian restaraunts located within the desired radius from Conrad Hotel; each variable in this list is a DICTIONARY containing all the relevant parameters - name, address, etc. - for each venue) into a more convenient format.

<b>json_normalize</b> method does just that! It takes a dict or a list of dicts and turn them into a pandas dataframe:

In [21]:
# tranforming json file into a pandas dataframe library
from pandas import json_normalize

In [22]:
df = json_normalize(query_results['response']['venues'])
df.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1610108733,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1610108733,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN
2,3fd66200f964a520f4e41ee3,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1610108733,False,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway


We want to keep only a relevant data: place's name; part of the 'categories' column that tells what type of a place it is (restaurant/pizza/pub/...); any information which is location related and place's ID that we are going to need later.

In [23]:
# Find all the columns having a word 'location' in their name:
location_data = []
for col in df.columns:
    if col.startswith('location'):
        location_data.append(col)
location_data

['location.address',
 'location.lat',
 'location.lng',
 'location.labeledLatLngs',
 'location.distance',
 'location.postalCode',
 'location.cc',
 'location.city',
 'location.state',
 'location.country',
 'location.formattedAddress',
 'location.crossStreet']

Remember that "+" operator applied on lists CONCATENATE these lists rather than summing the elements:

In [24]:
relevant_columns = ['name','categories']+location_data+['id']
df_formatted = df.loc[:,relevant_columns]
df_formatted

,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,id
0,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,4f3232e219836c91c7bfde94
2,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway,3fd66200f964a520f4e41ee3


In [25]:
# For better readability, we want to ommit the word 'location' in column names:
location_data_formatted = []
for col in location_data:
    location_data_formatted.append(col.split('.')[1])
location_data_formatted

['address',
 'lat',
 'lng',
 'labeledLatLngs',
 'distance',
 'postalCode',
 'cc',
 'city',
 'state',
 'country',
 'formattedAddress',
 'crossStreet']

In [26]:
df_formatted.columns = ['name','category']+location_data_formatted+['id']
df_formatted

,name,category,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,4f3232e219836c91c7bfde94
2,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway,3fd66200f964a520f4e41ee3


Finally we extract the venue type from the "categories" column and replace that column with the new column of the venues' types:

In [27]:
categories_formatted = []
for i in range(len(df['categories'])):
    for key in list(df['categories'][i][0].keys()):
        if key == 'name':
            categories_formatted.append(df['categories'][i][0][key])
categories_formatted

['Pizza Place', 'Food', 'Italian Restaurant']

In [28]:
df_formatted['category']=categories_formatted
df_formatted

,name,category,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway,3fd66200f964a520f4e41ee3


Let's visualize our location (red circle) and the suggested places:

In [29]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=12,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df_formatted.lat, df_formatted.lng, df_formatted.name):
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<h3>Explore the Suggested Venues</h3>

There is more than one suggestion inside of the desired radius. Hence we want to further explore each of those places in order to find the best one.

In [30]:
# Find the ID of "Harry's Italian Pizza Bar"
venue_id = df_formatted.loc[0,:][-1]
venue_id

'4fa862b3e4b0ebff2f749f06'

Next we are going to make a Venue Details API call in order to find the "Harry's Italian Pizza Bar" rating. Bote that this is a <b>Premium</b> endpoint - meaning we are only allowed to make 50 calls like that per day using a Sandbox account on FourSquare, or 500 - using their Personal free tier account.

In [31]:
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id,CLIENT_ID,CLIENT_SECRET,VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=BWN5AFL11Q15E005RDLMI4IL2YDPMGQILRXPMGDZJ10PWJOD&client_secret=KPYVQ0UIFPNWATAUU352BRTHN3RUVDN1C3THDDIHY3KBOKKE&v=20210103'

In [32]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
# Find the venue's rating:
result['response']['venue']['rating']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


6.9

This is not a very good grade... What about 'Ecco'?

In [114]:
venue_id = df_formatted.loc[2,:][-1]
venue_id

'3fd66200f964a520f4e41ee3'

In [115]:
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id,CLIENT_ID,CLIENT_SECRET,VERSION)
url

'https://api.foursquare.com/v2/venues/3fd66200f964a520f4e41ee3?client_id=BWN5AFL11Q15E005RDLMI4IL2YDPMGQILRXPMGDZJ10PWJOD&client_secret=KPYVQ0UIFPNWATAUU352BRTHN3RUVDN1C3THDDIHY3KBOKKE&v=20210103'

In [116]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']['rating']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


7.3

That's a better grade, thus we prefer 'Ecco'. Now let's try to find more about this place:

In [117]:
result['response']['venue']['tips']

{'count': 19,
 'groups': [{'type': 'others',
   'name': 'All tips',
   'count': 19,
   'items': [{'id': '4c7ac41f278eb713ae245c80',
     'createdAt': 1283114015,
     'text': 'Old world comfort where you can enjoy veal chop alla salvia, lobster fra diavolo and branzino al forno. You can get our free Downtown dining guide at http://www.downtownny.com/mapsguides/',
     'entities': [{'indices': [150, 187],
       'type': 'url',
       'object': {'url': 'http://www.downtownny.com/mapsguides/'}}],
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/4c7ac41f278eb713ae245c80',
     'lang': 'en',
     'likes': {'count': 2,
      'groups': [{'type': 'others',
        'count': 2,
        'items': [{'firstName': 'The Official Khalis', 'countryCode': 'US'}]}],
      'summary': '2 likes'},
     'logView': True,
     'agreeCount': 2,
     'disagreeCount': 0,
     'todo': {'count': 5},
     'user': {'firstName': 'Downtown',
      'lastName': 'A',
      'countryCode': 'US'}}]}]}

In [41]:
counts = result['response']['venue']['tips']['count']
counts

19

There are 19 tips overall. That's the number we will provide to an API call as a "limit" parameter. 

Let's view some[1] of those tips. We do it using Venue Tips endpoint:



[1] Remember that Sandbox account only allows us to view a <b>single</b> tip per venue!

In [121]:
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(venue_id,CLIENT_ID,CLIENT_SECRET, ACCESS_TOKEN, VERSION, counts)
url

'https://api.foursquare.com/v2/venues/3fd66200f964a520f4e41ee3/tips?client_id=BWN5AFL11Q15E005RDLMI4IL2YDPMGQILRXPMGDZJ10PWJOD&client_secret=KPYVQ0UIFPNWATAUU352BRTHN3RUVDN1C3THDDIHY3KBOKKE&oauth_token=T0LCIZDS5XSOIHNJUE3XAE32EN3TTTU5U1FHVTVE0UGTWRE5&v=20210103&limit=19'

In [122]:
tips = requests.get(url).json()
tips

{'meta': {'code': 200, 'requestId': '5ff86f8db16446703b2af477'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'tips': {'count': 18,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'likes': {'count': 0, 'groups': []},
     'like': False,
     'logView': True,
     'agreeCount': 5,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '484542633',
      'firstName': 'Nick',
      'lastName': 'El-Tawil',
      'gender': 'male',
      'countryCode': 'US',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/484542633_unymNUmw_FdPs3GjXHujmHcYnN4hf8kEPADlOZuIrdcdm97VX3tFqL7fFNM

In [123]:
tips['response']['tips']['items'][0]['text']

'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites'

We would like to get personal information for the user who left this review.

We extract this user's user id, then provide it to <i>User Details</i> endpoint:

In [127]:
user_id = tips['response']['tips']['items'][0]['user']['id']
user_id

'484542633'

In [130]:
url = 'https://api.foursquare.com/v2/users/{}?oauth_token={}&v={}'.format(user_id,ACCESS_TOKEN,VERSION)
url

'https://api.foursquare.com/v2/users/484542633?oauth_token=T0LCIZDS5XSOIHNJUE3XAE32EN3TTTU5U1FHVTVE0UGTWRE5&v=20210103'

In [131]:
user = requests.get(url).json()
#user

Let's print user's Nick El-Tawil's picture:

In [145]:
user['response']['user']['photo']

{'prefix': 'https://fastly.4sqi.net/img/user/',
 'suffix': '/484542633_unymNUmw_FdPs3GjXHujmHcYnN4hf8kEPADlOZuIrdcdm97VX3tFqL7fFNMNA_8Gl9NlU1GYg.jpg'}

We can create a .jpg image object from a url or a file path and display the image as follows:

In [155]:
import IPython

from IPython.core.display import Image
#from IPython.display import Image
from IPython.core.display import HTML

In [156]:
Image(url = 'https://fastly.4sqi.net/img/user/100x200/484542633_unymNUmw_FdPs3GjXHujmHcYnN4hf8kEPADlOZuIrdcdm97VX3tFqL7fFNMNA_8Gl9NlU1GYg.jpg')

Note that here we built the url from the 'prefix' concatenated with the 'suffix' using the desired image's size.